
Задание для курсового проекта

Метрика: R2 - коэффициент детерминации (sklearn.metrics.r2_score)
Сдача проекта:

    Срок сдачи: 05.02.2021
    Прислать в раздел Задания Урока 10 ("Вебинар. Консультация по итоговому проекту") ссылку на ноутбук в github или public kaggle notebook
    Необходимо получить R2 > 0.6 на Private Leaderboard
    Ник на kaggle - alena77

Примечание: Все файлы csv должны содержать названия полей (header - то есть "шапку"), разделитель - запятая. В файлах не должны содержаться индексы из датафрейма.
Рекомендации для файла с кодом (ipynb):

    Файл должен содержать заголовки и комментарии
    Повторяющиеся операции лучше оформлять в виде функций
    По возможности добавлять графики, описывающие данные (около 3-5)
    Добавлять только лучшую модель, то есть не включать в код все варианты решения проекта
    Скрипт проекта должен отрабатывать от начала и до конца (от загрузки данных до выгрузки предсказаний)
    Весь проект должен быть в одном скрипте (файл ipynb).
    При использовании статистик (среднее, медиана и т.д.) в качестве признаков, лучше считать их на трейне, и потом на валидационных и тестовых данных не считать статистики заново, а брать их с трейна.
    Проект должен полностью отрабатывать за разумное время (не больше 10 минут), поэтому в финальный вариант лучше не включать GridSearch с перебором большого количества сочетаний параметров.



Подключение библиотек и скриптов

In [64]:
import numpy as np 
import pandas as pd 
import os


for dirname, _, dirname in os.walk('C:/Users/vazhik/Documents/Data Science'):
    print(dirname, dirname)
    for filename in filenames:
        print(os.path.join(dirname, filename))
# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

['creditcard.csv'] ['creditcard.csv']


NameError: name 'filenames' is not defined

In [29]:
os.listdir('C:/Users/vazhik/Documents/Data Science')

['creditcard.csv', 'Python-Data-Science']

In [61]:
os.walk('C:/Users/vazhik/Documents/Data Science')

<generator object _walk at 0x0000021D1EC35820>

In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import random
import warnings

from matplotlib import style
from sklearn import ensemble
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score as r2
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import KFold, GridSearchCV
from datetime import datetime
from pylab import rcParams

style.use('fivethirtyeight')
%matplotlib inline
warnings.filterwarnings('ignore')

In [5]:
matplotlib.rcParams.update({'font.size': 14})

In [6]:
def evaluate_preds(train_true_values, train_pred_values, test_true_values, test_pred_values):
    print("Train R2:\t" + str(round(r2(train_true_values, train_pred_values), 3)))
    print("Test R2:\t" + str(round(r2(test_true_values, test_pred_values), 3)))
    
    plt.figure(figsize=(18,10))
    
    plt.subplot(121)
    sns.scatterplot(x=train_pred_values, y=train_true_values)
    plt.xlabel('Predicted values')
    plt.ylabel('True values')
    plt.title('Train sample prediction')
    
    plt.subplot(122)
    sns.scatterplot(x=test_pred_values, y=test_true_values)
    plt.xlabel('Predicted values')
    plt.ylabel('True values')
    plt.title('Test sample prediction')

    plt.show()

Загрузка данных

In [ ]:
TEST_DATASET_PATH = '/kaggle/input/realestatepriceprediction/test.csv'
TRAIN_DATASET_PATH = '/kaggle/input/realestatepriceprediction/train.csv'

test_data = pd.read_csv(TEST_DATASET_PATH)
train_data = pd.read_csv(TRAIN_DATASET_PATH)

Описание датасета

    Id - идентификационный номер квартиры
    DistrictId - идентификационный номер района
    Rooms - количество комнат
    Square - площадь
    LifeSquare - жилая площадь
    KitchenSquare - площадь кухни
    Floor - этаж
    HouseFloor - количество этажей в доме
    HouseYear - год постройки дома
    Ecology_1, Ecology_2, Ecology_3 - экологические показатели местности
    Social_1, Social_2, Social_3 - социальные показатели местности
    Healthcare_1, Helthcare_2 - показатели местности, связанные с охраной здоровья
    Shops_1, Shops_2 - показатели, связанные с наличием магазинов, торговых центров
    Price - цена квартиры

In [ ]:
train_data.tail()

Анализ распределения целевой переменной в разрезе иных признаков

In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(train_data['Rooms'], train_data['Price'])
plt.xlabel('Rooms')
plt.ylabel('Price')
plt.title('Distribution of Price by Rooms')
plt.show()

Исключение малозначимых столбцов, содержащих количественные признаки

In [ ]:
train_data.dtypes

In [ ]:
df_num_features = train_data.select_dtypes(include='float64')

num_features = pd.DataFrame(df_num_features)
num_features.drop('Ecology_1', axis=1, inplace=True)
num_features.drop('Healthcare_1', axis=1, inplace=True)
num_features.hist(figsize=(10, 8), bins=20, grid=False)

Взаимосвязь признаков

In [ ]:
corr = num_features.corr()
plt.figure(figsize = (8, 8))
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
sns.set(font_scale=1.4)
sns.heatmap(num_features.corr(), mask=mask, annot=True, fmt='.1f', linewidths=.5, cmap='GnBu')
plt.title('Relationship matrix')
plt.show()

In [ ]:
test_data.info()

Заполнение пропусков

In [ ]:
train_data = train_data.sort_values('Price')
test_data = test_data.sort_values('DistrictId')
train_data = train_data.fillna(method='pad')
test_data = test_data.fillna(method='pad')

Проверка данных

In [ ]:
rcParams['figure.figsize'] = 12, 6  # изменение размера графиков

plt.scatter(train_data.Price, train_data.Square)

Формирование и обучение модели. Предсказание цен

In [ ]:
# формирование модели

X_train = train_data.drop('Price', axis=1) # 1
y_train = train_data['Price'] # 1

X_test = test_data # 2

pr = pd.DataFrame()
pr['Id'] = X_test['Id'].copy() # 2.0

del_list = ["Id", "DistrictId", "LifeSquare", "Healthcare_1", "Ecology_2", "Ecology_3", "Shops_2"]
X_train.drop(del_list, axis=1, inplace=True) # 1
X_test.drop(del_list, axis=1, inplace=True) # 2

# обучение модели
model = RandomForestRegressor(n_estimators=1000, max_depth=16, random_state=42, max_features=7)
model.fit(X_train, y_train) # 1

y_pred = model.predict(X_test) # 2

# предсказание цен
pr['Price'] = y_pred # 2.0
pr.to_csv('predictions.csv', index=False) # 2.0